<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2024-01-15 20:06:12


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ', 'EQUITAS']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [5]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','Breakout','StarStock', 'Conviction', 'Category', 'Stock', 'MedianPE']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [6]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
1    71
0    63
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    23
V40     22
VR      12
AR       6
V200     1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    112
0     22
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1    28
0    17
Name: StarStock, dtype: int64
-------


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%Min'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

3MINDIA
ABBOTINDIA
ACC
ADANIENT
ADANIGREEN
ADANIPORTS
AKZOINDIA
ALKYLAMINE
AMBUJACEM
ANGELONE
APOLLOHOSP
ASTRAZEN
AUBANK
AUROPHARMA
AXISBANK
BAJAJ-AUTO
BAJAJHLDNG
BHARTIARTL
BIOCON
BLUESTARCO
BOSCHLTD
BPCL
CANBK
CAPLIPOINT
CASTROLIND
CDSL
CERA
CHOLAFIN
CIPLA
COALINDIA
COLPAL
DEEPAKNTR
DIVISLAB
DIXON
DLF
DREAMFOLKS
DRREDDY
EICHERMOT
EQUITASBNK
ERIS
FEDERALBNK
FINCABLES
GAIL
GILLETTE
GLAXO
GODREJCP
GODREJIND
GRASIM
HAVELLS
HCLTECH
HDFCAMC
HDFCLIFE
HEROMOTOCO
HINDALCO
HINDPETRO
IBREALEST
ICICIBANK
ICICIPRULI
IGL
INDIANB
INDIGO
INDUSTOWER
IOC
ISEC
ITBEES
ITC
J&KBANK
JCHAC
JINDALSTEL
JPPOWER
JSWSTEEL
LALPATHLAB
LT
LUPIN
M&M
MARUTI
MCDOWELL-N
MCX
MOTILALOFS
MUTHOOTFIN
NAM-INDIA
NATIONALUM
NAUKRI
NESTLEIND
NETWORK18
NIACL
NMDC
NTPC
OFSS
ONGC
PEL
PFIZER
PGHH
PIIND
PNB
POLYCAB
POWERGRID
PRAJIND
PVRINOX
RADICO
SANOFI
SBILIFE
SHREECEM
SIEMENS
SPORTKING
SRF
SUNPHARMA
SUZLON
TASTYBITE
TATACONSUM
TATAMOTORS
TATAPOWER
TATASTEEL
TCS
TECHM
TIPSINDLTD
TITAN
TORNTPHARM
TTKPRESTIG
UJJIVAN
ULTRACEMCO
UPL
V

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%Min,RSI_14
Date,,,,,,,,
2023-12-14 00:00:00+05:30,21183.0,18944.0,11.82,^NSEI,22097.45,16945.05,25.01,82.0
2023-12-21 00:00:00+05:30,21255.0,19039.0,11.64,^NSEI,22097.45,16945.05,25.44,70.0
2023-12-29 00:00:00+05:30,21731.0,19142.0,13.53,^NSEI,22097.45,16945.05,28.25,76.0
2024-01-08 00:00:00+05:30,21513.0,19248.0,11.77,^NSEI,22097.45,16945.05,26.96,62.0
2024-01-15 00:00:00+05:30,22097.0,19366.0,14.10,^NSEI,22097.45,16945.05,30.41,74.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [12]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [14]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
df_prospects['Dev%_PE'] = round((df_prospects['Curr_PE'] - df_prospects['MedianPE'])*100/df_prospects['MedianPE'],2)
#df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Max']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '200_SMA', 'Symbol', 'Target', 'Max', 'Min', 'Close', 'MedianPE', 'Curr_PE'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [15]:
# v40 and v40n
df_tmp = df_prospects[df_prospects['Criteria'].isin(['V40','V40N'])]
data_table.DataTable(df_tmp, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
3MINDIA.NS,20.89,60.70,57.0,31.40,23.40,V40N,ATH,1,NaN,1,M,MISC,-0.14,8.0
ABBOTINDIA.NS,14.87,31.67,81.0,41.10,31.60,V40,NTT,0,NaN,1,H,PHARMA,3.41,0.0
AKZOINDIA.NS,8.01,28.42,66.0,34.70,26.10,V40,NTT,1,NaN,1,H,NaN,-11.14,7.0
ANGELONE.NS,93.69,295.28,76.0,44.00,47.10,V40N,ATH,1,NaN,1,M,FINANCE,58.71,-0.0
ASTRAZEN.NS,49.66,98.38,68.0,30.90,23.00,V40N,ATH,0,NaN,1,M,PHARMA,-11.80,7.0
AXISBANK.NS,14.12,35.99,56.0,5.12,8.73,V40,ATH,1,NaN,1,H,BANKS,-49.48,2.0
BAJAJ-AUTO.NS,46.88,118.26,79.0,26.20,20.20,V40,ATH,1,NaN,1,H,AUTO,60.00,0.0
BAJAJHLDNG.NS,16.62,49.58,63.0,9.71,11.20,V40,ATH,1,NaN,0,H,FINANCE,27.91,1.0
BOSCHLTD.NS,19.85,44.71,73.0,17.40,13.20,V40N,ATH,1,NaN,1,M,AUTO,20.27,0.0


In [16]:
# all other prospects
df_tmp = df_prospects[~df_prospects['Criteria'].isin(['V40','V40N'])]
data_table.DataTable(df_tmp, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
ACC.NS,21.17,45.24,65.0,10.00,7.06,NaN,NaN,0,NaN,0,L,CEMENT,NaN,17.0
ADANIENT.NS,29.48,158.99,68.0,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,NaN,35.0
ADANIGREEN.NS,61.10,266.68,69.0,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,NaN,75.0
ADANIPORTS.NS,50.01,161.91,78.0,9.53,14.40,NaN,NaN,1,NaN,1,L,NaN,NaN,1.0
ALKYLAMINE.NS,1.93,14.83,49.0,41.30,41.40,NaN,NaN,0,NaN,0,L,CHEMICALS,NaN,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VGUARD.NS,3.07,21.06,46.0,15.20,12.60,NaN,NaN,1,NaN,1,L,NaN,NaN,9.0
VOLTAS.NS,26.74,43.92,80.0,9.67,4.42,NaN,NaN,1,NaN,0,L,AC,164.98,24.0
WSTCSTPAPR.NS,20.61,59.60,59.0,52.90,44.80,VR,ATH,0,NaN,0,L,PAPER,NaN,5.0


In [17]:
# top 10 prospects to invest from BTT stocks
df_prospects[df_prospects['Strategy'] == 'BTT'].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
CDSL.NS,41.44,111.31,51.0,30.30,23.8,VR,BTT,1,NaN,1,L,MISC,65.84,6.0
CDSL.NS,41.44,111.31,51.0,30.30,23.8,VR,BTT,1,NaN,1,L,MISC,65.84,6.0
HDFCAMC.NS,43.59,114.51,77.0,1000.00,1000.0,V40,BTT,1,NaN,1,H,FINANCE,2425.25,0.0
HINDPETRO.NS,57.20,109.80,71.0,-8.08,-18.9,VR,BTT,1,260.0,1,L,REFINERIES,-21.63,0.0


In [18]:
# prospects to invest from star + V40/V40N/VR + below 200 DMA
df_prospects[(df_prospects['Criteria'].notnull()) & (df_prospects['StarStock'] == 1) & (df_prospects['Dev%_200'] < 5)].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
POLYCAB.NS,-5.06,57.92,28.0,27.0,20.0,V40N,ATH,1,NaN,1,M,CABLES,8.59,36.0
CERA.NS,1.97,53.81,51.0,25.5,19.4,V40N,ATH,1,NaN,1,M,CERAMICS,9.95,19.0


In [19]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
ADANIGREEN.NS,61.10,266.68,69.0,7.81,22.8,NaN,NaN,1,NaN,1,L,NaN,NaN,75.0
TASTYBITE.NS,0.94,73.01,42.0,1000.00,1000.0,NaN,NaN,0,NaN,1,L,NaN,NaN,47.0
BLUESTARCO.NS,28.61,75.99,76.0,24.40,23.4,AR,NTT,1,NaN,1,L,AC,36.09,42.0
POLYCAB.NS,-5.06,57.92,28.0,27.00,20.0,V40N,ATH,1,NaN,1,M,CABLES,8.59,36.0
IGL.NS,-2.96,14.72,63.0,26.60,20.5,NaN,NaN,1,NaN,1,L,NaN,NaN,29.0
PRAJIND.NS,11.85,75.40,37.0,31.00,23.7,AR,ATH,1,NaN,1,L,ENGINEERING,NaN,23.0
CERA.NS,1.97,53.81,51.0,25.50,19.4,V40N,ATH,1,NaN,1,M,CERAMICS,9.95,19.0
PIIND.NS,-1.85,19.39,46.0,21.70,18.5,NaN,NaN,1,NaN,1,L,NaN,NaN,15.0
MUTHOOTFIN.NS,16.41,62.04,48.0,12.10,17.9,NaN,NaN,1,NaN,1,L,FINANCE,16.80,14.0


In [20]:
# top 10 prospects to invest from  V40/V40N/VR
df_prospects[df_prospects['Criteria'].notnull()].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
NESTLEIND.NS,-88.12,0.00,9.0,138.0,108.00,V40,ATH,0,NaN,0,H,FMCG,8.15,943.0
JCHAC.NS,10.10,32.00,64.0,1000.0,1000.00,V40N,ATH,0,NaN,0,M,AC,1823.08,159.0
LALPATHLAB.NS,8.60,40.32,41.0,18.2,15.00,V40N,ATH,1,NaN,0,M,HEALTHCARE,0.88,68.0
TTKPRESTIG.NS,2.61,18.83,62.0,18.1,13.80,V40N,ATH,1,NaN,0,M,DURABLES,11.26,49.0
DREAMFOLKS.NS,1.86,52.15,54.0,76.8,60.20,AR,ATH,0,NaN,0,L,MISC,NaN,47.0
UPL.NS,-9.45,6.71,43.0,14.3,13.40,AR,NTT,1,NaN,0,L,NaN,NaN,44.0
BLUESTARCO.NS,28.61,75.99,76.0,24.4,23.40,AR,NTT,1,NaN,1,L,AC,36.09,42.0
PFIZER.NS,11.56,26.83,66.0,27.2,19.80,V40,ATH,0,NaN,0,H,PHARMA,256.16,38.0
ICICIPRULI.NS,-0.15,36.11,39.0,10.0,8.45,V40,ATH,1,NaN,0,H,INSURANCE,11.88,37.0


In [21]:
# top 10 prospects to invest below 200 DMA
df_prospects[df_prospects['Dev%_200'] < 0].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
NESTLEIND.NS,-88.12,0.00,9.0,138.00,108.00,V40,ATH,0,NaN,0,H,FMCG,8.15,943.0
UPL.NS,-9.45,6.71,43.0,14.30,13.40,AR,NTT,1,NaN,0,L,NaN,NaN,44.0
ICICIPRULI.NS,-0.15,36.11,39.0,10.00,8.45,V40,ATH,1,NaN,0,H,INSURANCE,11.88,37.0
POLYCAB.NS,-5.06,57.92,28.0,27.00,20.00,V40N,ATH,1,NaN,1,M,CABLES,8.59,36.0
VINATIORGA.NS,-5.28,1.71,47.0,30.40,22.60,V40N,ATH,1,NaN,0,M,CHEMICALS,1.64,36.0
VIPIND.NS,-8.57,3.02,38.0,27.80,29.00,V40N,ATH,1,NaN,0,M,MISC,16.37,30.0
IGL.NS,-2.96,14.72,63.0,26.60,20.50,NaN,NaN,1,NaN,1,L,NaN,NaN,29.0
PVRINOX.NS,-5.12,12.26,26.0,1000.00,1000.00,NaN,NaN,1,NaN,0,L,NaN,1853.12,27.0
HDFCLIFE.NS,-0.57,33.61,31.0,9.56,9.57,V40,ATH,1,NaN,0,H,INSURANCE,-3.02,23.0


In [22]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 19


,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
ABBOTINDIA.NS,14.87,31.67,81.0,41.10,31.60,V40,NTT,0,NaN,1,H,PHARMA,3.41,0.0
OFSS.NS,31.02,78.41,79.0,35.10,24.80,V40N,ATH,1,NaN,1,M,IT,43.98,-0.0
HDFCAMC.NS,43.59,114.51,77.0,1000.00,1000.00,V40,BTT,1,NaN,1,H,FINANCE,2425.25,0.0
HCLTECH.NS,32.24,58.54,74.0,28.30,23.00,V40,ATH,1,NaN,1,H,IT,39.09,0.0
TCS.NS,14.78,27.95,67.0,58.70,46.90,V40,ATH,1,3144.0,1,H,IT,11.74,-0.0
TITAN.NS,21.49,63.38,70.0,25.10,30.80,V40,ATH,1,NaN,1,H,JEWELLERY,29.49,-0.0
BOSCHLTD.NS,19.85,44.71,73.0,17.40,13.20,V40N,ATH,1,NaN,1,M,AUTO,20.27,0.0
BAJAJ-AUTO.NS,46.88,118.26,79.0,26.20,20.20,V40,ATH,1,NaN,1,H,AUTO,60.00,0.0
ANGELONE.NS,93.69,295.28,76.0,44.00,47.10,V40N,ATH,1,NaN,1,M,FINANCE,58.71,-0.0


In [23]:
# prospects for interactive analysis

data_table.DataTable(df_prospects, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
3MINDIA.NS,20.89,60.70,57.0,31.40,23.40,V40N,ATH,1,NaN,1,M,MISC,-0.14,8.0
ABBOTINDIA.NS,14.87,31.67,81.0,41.10,31.60,V40,NTT,0,NaN,1,H,PHARMA,3.41,0.0
ACC.NS,21.17,45.24,65.0,10.00,7.06,NaN,NaN,0,NaN,0,L,CEMENT,NaN,17.0
ADANIENT.NS,29.48,158.99,68.0,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,NaN,35.0
ADANIGREEN.NS,61.10,266.68,69.0,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,NaN,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VIPIND.NS,-8.57,3.02,38.0,27.80,29.00,V40N,ATH,1,NaN,0,M,MISC,16.37,30.0
VOLTAS.NS,26.74,43.92,80.0,9.67,4.42,NaN,NaN,1,NaN,0,L,AC,164.98,24.0
WSTCSTPAPR.NS,20.61,59.60,59.0,52.90,44.80,VR,ATH,0,NaN,0,L,PAPER,NaN,5.0


In [24]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [25]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%Min,RSI_14
Date,,,,,,,,
2023-12-15 00:00:00+05:30,586.0,549.0,6.70,BERGEPAINT.NS,710.11,447.11,31.02,56.0
2023-12-22 00:00:00+05:30,583.0,552.0,5.64,BERGEPAINT.NS,710.11,447.11,30.43,53.0
2024-01-01 00:00:00+05:30,604.0,554.0,9.00,BERGEPAINT.NS,710.11,447.11,35.06,62.0
2024-01-08 00:00:00+05:30,579.0,557.0,3.92,BERGEPAINT.NS,710.11,447.11,29.46,44.0
2024-01-15 00:00:00+05:30,603.0,560.0,7.63,BERGEPAINT.NS,710.11,447.11,34.81,59.0
